In [2]:
import pandas as pd
from sklearn import ensemble
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

df = pd.read_csv('NCAA_Tourney_2002-2016_1.csv')
features = df.iloc[:, :-4]
results = df.iloc[:, -4:]

X = features
y = results.loc[:, 'result']
X_train = pd.DataFrame()
X_test = pd.DataFrame()
y_train = pd.DataFrame()
y_test = pd.DataFrame()
X_valid = pd.DataFrame()
y_valid = pd.DataFrame()


for year in range(2002, 2014):
    X_train = pd.concat([X_train, X.loc[results.Season == year]])
    y_train = pd.concat([y_train, y.loc[results.Season == year]])


for year in range(2014, 2017):
    X_test = pd.concat([X_test, X.loc[results.Season == year]])
    y_test = pd.concat([y_test, y.loc[results.Season == year]])
    
paramsGB = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2, 'learning_rate': 0.01, 'loss': 'ls'}

clf = ensemble.GradientBoostingRegressor(**paramsGB)

clf.fit(X_train, y_train.values.ravel())

y_pred = clf.predict(X_test)

sns.set(font_scale=1, rc={"lines.linewidth": 1.2})

fig = plt.figure(figsize=(20, 10), dpi=80)
fig.suptitle('Predictor Importance', fontsize=20)

plt.subplot(1,1,1)

f = clf.feature_importances_

f = 100.0 * (f / f.max())
indices = np.argsort(f)[-10:]
plt.barh(np.arange(10), f[indices],color='dodgerblue',alpha=.4)
plt.yticks(np.arange(10 + 0.25), np.array(X.columns)[indices])
plt.xlabel('Relative importance'), \
plt.title('Top Ten Important Variables')

plt.show()